# Python in a nutshell - Probability and statistics

_Most of this content is based off of David Biersach's [SciComp101 course](https://github.com/dbiersach/scicomp101) on GitHub, check it out!_

This notebook contains multiple code-snippets which you will likely work through with an instructor. You're encouraged to run these cells yourself, modify the code, and experiment!

**Import packages used in this notebook**

🚨 Note: you might have to install `numba` via `!pip install numba`, which you can do right from the notebook.

In [ ]:
import collections
import numpy as np
import matplotlib.pyplot as plt
import scipy
from numba import njit

# Basic statistics

**Set the numpy random number `seed` to 2016**

In [ ]:
np.random.seed(2016)

**Create and display an array $a$ of 30 integers where each element is in the range [0,100]**

In [ ]:
a = np.random.randint(1, 101, 30)
a

**Define a function `mean(s)` that returns the average of the values in the array $s$**\
then print the value returned by that function when passed array $a$

In [ ]:
def mean(s):
    return np.sum(s) / len(s)

print(f"{mean(a) = }")

**Define a function `median(s)` that returns the median value of the array $s$**\
then print the value returned by that function when passed array $a$\
NOTE: The array $s$ may have an <u>even</u> number of items!

In [ ]:
def median(s):
    s.sort()
    i = len(s) // 2
    if len(s) % 2 == 1: # s has an odd number of elements
        return s[i]
    else: # s has an even number of elements
        return (s[i - 1] + s[i]) / 2

print(f"{median(a) = }")

**Tally the total occurrences of each number in array $a$**

In [ ]:
collections.Counter(a)

**Define a function `mode(s)` that returns the mode value of the array $s$**\
then print the value returned by that function when passed array $a$\
NOTE: The array $s$ may be <u>multi-modal</u>!

In [ ]:
def mode(s):
    counts = collections.Counter(s)
    max_count = max(counts.values())
    if max_count == 1:
        return None
    else:
        return [k for k, v in counts.items() if v == max_count]

print(f"{mode(a) = }")

# Hero abilities

**Set $n$ (the number of rolls) to be 1,000,000**

**Simulate $n$ number of 3d8 rolls and store each roll in array $a$**

**Display the first ten 3d8 rolls**

**Simulate $n$ number of 1d20 rolls and store each roll in array $b$**

**Display the first ten 1d20 rolls**

**Print the mean $(\mu)$ and standard deviation $(\sigma)$ across all 3d8 rolls**

**Print the mean $(\mu)$ and standard deviation $(\sigma)$ across all 1d20 rolls**

# Uniform variance

**Define a function `run_trial(trial_num)` that**: \
1. Creates a random array
2. Computes the magic number $\frac{(upper\ limit-lower\ limit)^2}{\sigma^2}$
3. Prints the various statistics for this trial

In [ ]:
def run_trial(trial_num):
    lower_limit = np.random.randint(10_001)
    upper_limit = lower_limit + np.random.randint(100_001)
    size = np.random.randint(10_000, 200_001)
    a = np.random.randint(lower_limit, upper_limit, size)
    mean, var = np.mean(a), np.var(a)
    magic = (upper_limit - lower_limit) ** 2 / var
    print(f"{trial_num:>8}", end="")
    print(f"{lower_limit:>9,}", end="")
    print(f"{upper_limit:>9,}", end="")
    print(f"{size:>9,}", end="")
    print(f"{mean:>14.3f}", end="")
    print(f"{var:>16.3f}", end="")
    print(f"{magic:>10.3f}")

**Print the table headers then run 15 trials of this experiment**

In [ ]:
print(f"{'Trial #':>8}", end="")
print(f"{'Lower':>9}", end="")
print(f"{'Upper':>9}", end="")
print(f"{'Size':>9}", end="")
print(f"{'Mean':>14}", end="")
print(f"{'Variance':>16}", end="")
print(f"{'Magic':>10}")

for trial_num in range(1, 16):
    run_trial(trial_num)

# Pachinko machine

**Define a `numba` accelerated function to simulate dropping $num\_balls$ through $num\_levels$ down the pachinko machine** \
The function returns an array $balls$ which contains the final slot number of each ball

In [ ]:
@njit
def pachinko_normal(num_balls, num_levels):
    np.random.seed(2016)
    balls = np.zeros(num_balls)
    for ball_num in range(num_balls):
        slot = 0
        for _ in range(num_levels):
            slot += -1 if np.random.rand() < 0.5 else 1
        balls[ball_num] = slot // 2
    return balls

**Define a function to graphically compare the distribution of balls dropped through a pachinko game \
with the exact (analytic) Gaussian standard normal distribution**

In [ ]:
def run_simulation(total_balls, total_levels):
    # Simulate the pachinko machine
    balls = pachinko_normal(total_balls, total_levels)

    # Calculate the mean number of of balls in each slot
    slots = np.zeros(total_levels + 1)
    first_slot = total_levels // 2
    for ball_num in range(total_balls):
        slot_num = int(first_slot + balls.take(ball_num))
        slots[slot_num] += 1
    slots = slots / total_balls

    # Create an array of slot numbers
    x = np.linspace(-total_levels // 2, total_levels // 2, total_levels + 1)

    # Calculate the expected number of balls in each slot
    mu = np.mean(balls)
    sigma = np.std(balls)

    # Create arrays to hold the values of a perfect normal distribution
    norm_x = np.linspace(-total_levels // 2, total_levels // 2, 100)
    norm_y = scipy.stats.norm(mu, sigma).pdf(norm_x)

    # Plot the results
    plt.plot(x, slots, color="blue", linewidth=2, label="Pachinko PDF")
    plt.plot(norm_x, norm_y, color="red", linewidth=2, label="Normal PDF")
    plt.title(
        f"Pachinko vs. Normal PDF ({total_balls:,} balls : {total_levels:,} levels)"
    )
    plt.xlabel("Slot Number")
    plt.ylabel("Probability")
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
run_simulation(total_balls=1_000, total_levels=10)

In [ ]:
run_simulation(total_balls=10_000, total_levels=10)

In [ ]:
run_simulation(total_balls=100_000, total_levels=20)

In [ ]:
run_simulation(total_balls=1_000_000, total_levels=40)

In [ ]:
run_simulation(total_balls=1_000_000, total_levels=80)

# Lattice circle

**Define a `numba` accelerated function to return the number of lattice points within a circle of given radius $r$**

**Demonstrate the veracity of Gauss's claim regarding circle lattice points**
1. For ten circles of increasing radius, print the <u>actual</u> and *estimated* number of lattice points
2. For each circle, also print the **percent relative error** in Gauss's estimate

In [ ]:
for radius in np.linspace(1000, 10000, 10):
    act = lattice_points(radius)
    est = int(np.pi * radius**2 + 2 * np.sqrt(2 * np.pi * radius))
    err = (est - act) / act
    print(
        f"Radius = {int(radius):>6,}"
        f"  Act Points = {act:>12,}"
        f"  Est Points = {est:>12,}"
        f"  % Rel. Err = {err:0.4%}"
    )

# Maxwell-Boltzmann

**Define the PDF of the [Maxwell-Boltzmann distribution](https://en.wikipedia.org/wiki/Maxwell%E2%80%93Boltzmann_distribution) with rate parameter $a$:**

$$\sqrt{\frac{2}{\pi}}\frac{x^2}{a^3}\ \mathrm{exp}\left(\frac{-x^2}{2a^2}\right)$$

In [ ]:
def pdf(x, a):
    # code here

**Set the domain $0\le x \le 20$**

In [ ]:
x = # code here

**Plot three PDF curves for rate parameters: $a=1$, $a=2$, $a=5$**

In [ ]:
plt.plot(x, pdf(x, 1)
plt.plot(x, pdf(x, 2)
plt.plot(x, pdf(x, 3)
plt.title("Maxwell-Boltzmann PDF")
plt.xlabel("x")
plt.ylabel("P(x)")
plt.legend(
plt.grid(
plt.show()